In [45]:
import pandas as pd

In [46]:
df = pd.read_csv('12_6_8-Dados.csv')

df

,millis,lat,lon,sat,alt,data,hora,altp,p,ax,ay,az,gx,gy,gz
0,2159,-22.286898,-42.542294,8,861.8,2025/5/15,12:6:9,-0.36,927.76,0.60,-0.01,9.03,-0.01,-0.02,0.02
1,2359,-22.286898,-42.542292,8,861.8,2025/5/15,12:6:10,-0.24,927.75,0.57,0.06,9.04,-0.01,-0.03,0.03
2,2559,-22.286898,-42.542292,8,862.0,2025/5/15,12:6:10,-0.36,927.76,0.60,0.03,8.86,-0.01,-0.02,0.02
3,2759,-22.286898,-42.542292,8,862.0,2025/5/15,12:6:10,-0.47,927.77,0.50,0.02,9.08,-0.01,-0.02,0.02
4,2959,-22.286898,-42.542292,8,862.0,2025/5/15,12:6:10,-0.28,927.75,0.58,0.06,9.09,-0.01,-0.02,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,142228,-22.286816,-42.542276,8,874.8,2025/5/15,12:8:29,-0.69,927.80,1.02,0.01,9.06,-0.01,-0.02,0.03
701,142428,-22.286814,-42.542274,8,874.9,2025/5/15,12:8:30,-0.44,927.77,1.06,0.06,8.99,-0.01,-0.01,0.02
702,142628,-22.286814,-42.542274,8,874.9,2025/5/15,12:8:30,-0.51,927.78,1.04,0.02,8.96,-0.01,-0.01,0.02
703,142828,-22.286814,-42.542274,8,874.9,2025/5/15,12:8:30,-0.44,927.77,1.04,0.00,9.05,-0.01,-0.02,0.02


In [ ]:
tempo_ativo = df['millis'].iloc[-1] - df['millis'].iloc[0]
tempo_ativo = tempo_ativo / 60000 # Tempo ativo em minutos

print(f'Tempo ativo: {tempo_ativo:.2f} minutos')

Tempo ativo: 140869 milissegundos
Tempo ativo: 2.35 minutos
